# Bronze to Silver Layer

Cleans and validates data from bronze layer:
- Removes duplicates
- Standardizes emails and phone numbers
- Separates valid records from invalid (quarantined) records
- Writes to silver layer with valid/invalid splits

In [0]:
customers_df = spark.read.format("delta").load("/mnt/bronze/customers/")
orders_df = spark.read.format("delta").load("/mnt/bronze/orders/")
products_df = spark.read.format("delta").load("/mnt/bronze/products/")

In [0]:
from pyspark.sql.functions import (
    lower, regexp_replace, col, when, lit, length
)

# Drop duplicates
products_df = products_df.dropDuplicates(["product_id"])
orders_df = orders_df.dropDuplicates(["order_id"])
customers_df = customers_df.dropDuplicates(["customer_id"])

In [0]:
# Standardize email
customers_df = customers_df.withColumn("email", lower(col("email")))

# Replace emails with no domain
customers_df = customers_df.withColumn(
    "email",
    when(~col("email").rlike(r"^[^@]+@[^@]+\.[^@]{2,}$"), lit(None))
    .otherwise(col("email"))
)

In [0]:
# Standardize phone numbers
customers_df = customers_df.withColumn("phone", regexp_replace(col("phone"), r"[^0-9]+", ""))
customers_df = customers_df.withColumn("phone", regexp_replace(col("phone"), r"^1", ""))

# Replace invalid phone numbers with null
customers_df = customers_df.withColumn("phone", when(length(col("phone")) < 10, lit(None)).otherwise(col("phone")))

In [0]:
# Define which rows should be quarantined
customers_invalid = customers_df.filter(col("customer_id").isNull())
orders_invalid = orders_df.filter(col("order_id").isNull())
products_invalid = products_df.filter(col("product_id").isNull())

customers_valid = customers_df.filter(col("customer_id").isNotNull())
orders_valid = orders_df.filter(col("order_id").isNotNull())
products_valid = products_df.filter(col("product_id").isNotNull())

In [0]:
# ----------------------------
# Write tables to correct path
# ----------------------------

# Define dataframes with corresponding names
datasets = [
    ("customers_valid", customers_valid),
    ("orders_valid", orders_valid),
    ("products_valid", products_valid),
    ("customers_invalid", customers_invalid),
    ("orders_invalid", orders_invalid),
    ("products_invalid", products_invalid)
]

for table_name, df in datasets:
    df.write \
        .mode("append") \
        .format("delta") \
        .save(f"/mnt/silver/{table_name}")